In [294]:
leo_preproc(leo)

/tmp/ipykernel_288286/235027801.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['certs_held'] == 'Y', 'certs_held'], df.loc[df['certs_held'] == 'N', 'certs_held'] = 1, 0 # Y -> 1, N -> 0
/tmp/ipykernel_288286/235027801.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[df['dprt_apt_id'] == 'NONE', 'dprt_apt_id'], df.loc[df['dprt_apt_id'] == 'PVT', 'dprt_apt_id'] = 0, 0 # None and PVT -> 0
/tmp/ipykernel_288286/235027801.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable

,certs_held,dprt_apt_id,dest_apt_id,flt_plan_filed,pc_profession,eng_type,carb_fuel_injection,NONE,IFR,CVFR,VFR
0,1,1,1,NONE,0,REC,CARB,0.0,0.0,1.0,0.0
1,1,1,1,NONE,0,REC,CARB,0.0,0.0,1.0,0.0
2,1,1,1,NONE,0,REC,FINJ,0.0,0.0,1.0,0.0
3,1,1,1,IFR,0,TP,UNK,0.0,1.0,0.0,0.0
4,1,1,1,NONE,0,REC,CARB,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
18802,1,1,0,NONE,0,REC,CARB,0.0,0.0,1.0,0.0
18803,1,1,1,IFR,0,REC,FINJ,0.0,1.0,0.0,0.0
18804,1,1,1,NONE,1,REC,CARB,0.0,0.0,1.0,0.0
18805,1,0,0,NONE,1,TS,UNK,0.0,0.0,1.0,0.0


In [14]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, Binarizer

In [292]:
data = pd.read_csv('../raw_data/wingman_data_cleaned.csv')
data.head(5)

,id,eventsoe_no,far_part,acft_make,acft_model,fixed_retractable,acft_category,homebuilt,flight_hours_mean,crew_category,...,crew_sex,certs_held,dprt_apt_id,dest_apt_id,flt_plan_filed,pc_profession,eng_type,carb_fuel_injection,type_fly,eng_mfgr
0,20080107X000261,250,091,PIPER,PA 28-180,FIXD,AIR,N,705.0,PLT,...,M,Y,O69,OQ3,NONE,No,REC,CARB,PERS,Textron Lycoming
1,20080107X000262,250,091,Barnard/Stancil,Glastar,FIXD,AIR,Y,8300.0,PLT,...,M,Y,OQ3,O70,NONE,No,REC,CARB,PERS,Textron Lycoming
2,20080109X000361,230,091,Micco Aircraft Company,MAC-145B,RETR,AIR,N,1000.0,PLT,...,F,Y,T20,AXH,NONE,No,REC,FINJ,PERS,Lycoming
3,20080107X000271,81,091,Pilatus,PC-12/45,RETR,AIR,N,10571.0,PLT,...,M,Y,PHX,PWA,IFR,UNK,TP,UNK,PERS,Pratt & Whitney Canada
4,20080115X000511,240,091,Cessna,152,FIXD,AIR,N,18.0,DSTU,...,M,Y,KFNL,KFNL,NONE,No,REC,CARB,INST,Lycoming


In [293]:
leo = data[['certs_held', 'dprt_apt_id', 'dest_apt_id',
            'flt_plan_filed', 'pc_profession', 'eng_type',
            'carb_fuel_injection']]
leo

,certs_held,dprt_apt_id,dest_apt_id,flt_plan_filed,pc_profession,eng_type,carb_fuel_injection
0,Y,O69,OQ3,NONE,No,REC,CARB
1,Y,OQ3,O70,NONE,No,REC,CARB
2,Y,T20,AXH,NONE,No,REC,FINJ
3,Y,PHX,PWA,IFR,UNK,TP,UNK
4,Y,KFNL,KFNL,NONE,No,REC,CARB
...,...,...,...,...,...,...,...
18802,Y,2CO5,PVT,NONE,No,REC,CARB
18803,Y,ATW,LOT,IFR,No,REC,FINJ
18804,Y,3L2,KVGT,NONE,Yes,REC,CARB
18805,Y,NONE,NONE,NONE,Yes,TS,UNK


In [268]:
def leo_preproc(df=data):
    """Returns processed df"""
    
    from sklearn.preprocessing import OneHotEncoder
    
    df.loc[df['certs_held'] == 'Y', 'certs_held'], df.loc[df['certs_held'] == 'N', 'certs_held'] = 1, 0 # Y -> 1, N -> 0
    
    df.loc[df['dprt_apt_id'] == 'NONE', 'dprt_apt_id'], df.loc[df['dprt_apt_id'] == 'PVT', 'dprt_apt_id'] = 0, 0 # None and PVT -> 0
    df.loc[df['dprt_apt_id'] !=0, 'dprt_apt_id'] = 1 # values != 0 -> 1
    
    df.loc[df['dest_apt_id'] == 'NONE', 'dest_apt_id'], df.loc[df['dest_apt_id'] == 'PVT', 'dest_apt_id'] = 0, 0 # None and PVT -> 0
    df.loc[df['dest_apt_id'] !=0, 'dest_apt_id'] = 1 # values != 0 -> 1
    
    df['flt_plan_filed'].replace('UNK', 'NONE', inplace=True)
    df['flt_plan_filed'].replace('VFIF', 'IFR', inplace=True)
    df['flt_plan_filed'].replace(['CFR', 'MFR'], 'VFR', inplace=True)
    df['flt_plan_filed'].replace(['VFIF', 'MVFR'], 'NONE',inplace=True)
    
    cat = list(df['flt_plan_filed'].unique())
    
    ohe = OneHotEncoder()
    ohe_df = pd.DataFrame(ohe.fit_transform(df[['flt_plan_filed']]).toarray())
    ohe_df.columns = cat
    ohe_df
    df.drop(columns='flt_plan_filed')
    df = pd.concat([df, ohe_df], axis=1)

    df['pc_profession'].replace('UNK', 'No', inplace=True)
    df['pc_profession'].replace(['Yes', 'No'], [1, 0], inplace=True)
    return df

In [267]:
leo['pc_profession'].value_counts()

pc_profession
No     11589
Yes     6119
UNK     1099
Name: count, dtype: int64

In [13]:
leo['certs_held'].value_counts()

certs_held
Y    18400
N      407
Name: count, dtype: int64

In [22]:
leo.loc[leo['certs_held'] == 'Y', 'certs_held'], leo.loc[leo['certs_held'] == 'N', 'certs_held'] = 1, 0

/tmp/ipykernel_288286/3923426174.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leo.loc[leo['certs_held'] == 'Y', 'certs_held'], leo.loc[leo['certs_held'] == 'N', 'certs_held'] = 1, 0


In [27]:
leo.head(5)

,certs_held,dprt_apt_id,dest_apt_id,flt_plan_filed,pc_profession,eng_type,carb_fuel_injection
0,1,O69,OQ3,NONE,No,REC,CARB
1,1,OQ3,O70,NONE,No,REC,CARB
2,1,T20,AXH,NONE,No,REC,FINJ
3,1,PHX,PWA,IFR,UNK,TP,UNK
4,1,KFNL,KFNL,NONE,No,REC,CARB


In [104]:
leo.pc_profession.value_counts()

# NO + UNK -> 0
# Yes -> 1

pc_profession
No     11589
Yes     6119
UNK     1099
Name: count, dtype: int64

In [224]:
leo['flt_plan_filed']

0        NONE
1        NONE
2        NONE
3         IFR
4        NONE
         ... 
18802    NONE
18803     IFR
18804    NONE
18805    NONE
18806    NONE
Name: flt_plan_filed, Length: 18807, dtype: object

In [253]:
cat = list(leo['flt_plan_filed'].unique())
ohe = OneHotEncoder()

test = ohe.fit_transform(leo[['flt_plan_filed']])
ohe_df = pd.DataFrame(ohe.transform(leo[['flt_plan_filed']]).toarray())

ohe_df.columns = cat
ohe_df


,NONE,IFR,CVFR,VFR
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
18802,0.0,0.0,1.0,0.0
18803,0.0,1.0,0.0,0.0
18804,0.0,0.0,1.0,0.0
18805,0.0,0.0,1.0,0.0


AttributeError: 'csr_matrix' object has no attribute 'tonumpy'

In [219]:
leo[['flt_plan_filed']]

,flt_plan_filed
0,NONE
1,NONE
2,NONE
3,IFR
4,NONE
...,...
18802,NONE
18803,IFR
18804,NONE
18805,NONE
